### Environmental Data Generation

In absence of data from the real project, I'm generating hypothetical indoor temperatures based on historical weather data containing air temperatures. The data is then saved as a CSV file for bulk upload to Supabase.

In [7]:
import pandas as pd
import numpy as np
import datetime

#### Load raw historical weather data and convert date field strings to datetime (numpy datetime64) objects

In [8]:
df = pd.read_csv('../../data/weather_data_0809.csv')
df['DATE'] = pd.to_datetime(df['DATE'])
print(df.head())

       STATION                                               NAME  LATITUDE  \
0  USW00094290  SEATTLE SAND POINT WEATHER FORECAST OFFICE, WA US   47.6872   
1  USW00094290  SEATTLE SAND POINT WEATHER FORECAST OFFICE, WA US   47.6872   
2  USW00094290  SEATTLE SAND POINT WEATHER FORECAST OFFICE, WA US   47.6872   
3  USW00094290  SEATTLE SAND POINT WEATHER FORECAST OFFICE, WA US   47.6872   
4  USW00094290  SEATTLE SAND POINT WEATHER FORECAST OFFICE, WA US   47.6872   

   LONGITUDE  ELEVATION       DATE  TAVG  TMAX  TMIN  TOBS  
0  -122.2553       18.3 2024-08-01   NaN    83    61   NaN  
1  -122.2553       18.3 2024-08-02   NaN    87    63   NaN  
2  -122.2553       18.3 2024-08-03   NaN    85    58   NaN  
3  -122.2553       18.3 2024-08-04   NaN    88    58   NaN  
4  -122.2553       18.3 2024-08-05   NaN    81    58   NaN  


#### Define conveyance operation times and peak temperature time

The conveyance in the warehouse runs for three different hour-long sessions during the day which generates high noise. I'm selecting an hour during the day which simulates when the peak temperature might occur for the given time of year.

In [9]:
peak_temperature_hour = 15  # 3 pm

conveyance_times = [
    (9, 10),    # Morning shift: 9 - 10 am
    (12, 13),   # Midday shift: 12 - 1 pm
    (15, 16)    # Afternoon shift: 3 - 4 pm
]

#### Create caps for temperature and noise levels

To simulate data readings after the installation of air conditioning and warehouse noise reductions, employ these caps.

In [10]:
use_caps = True

temperature_cap = 79.0
noise_cap = 59.0

#### Create simulated hourly readings

In [ ]:
simulated_data = []

for index, row in df.iterrows():
    date = row['DATE']
    tmax = float(row['TMAX'])
    tmin = float(row['TMIN'])

    # generate hourly temperature readings for operating hours (8 am to 5 pm)

    for hour_value in range(8, 18):
        timestamp = timestamp = datetime.datetime.combine(date, datetime.time(hour=hour_value))

        # Parabolic temp calculation (Thanks, AI.)
        temperature = tmin + (tmax - tmin) * (1 - ((hour_value - peak_temperature_hour) / (peak_temperature_hour - 8))**2)
        
        # While outside air temperatures can fall outside the range, it's never colder than 60 or hotter than 90 inside
        if temperature < 60:
            temperature = 60 + np.random.uniform(0, 0.5)
        elif temperature > 90:
            temperature = 90 - np.random.uniform(0, 0.5)
        
        if use_caps and temperature >= temperature_cap:
            temperature = temperature_cap 
        
        temperature += np.random.uniform(-0.5, 0.5)  # Add slight randomness for realism
        
        if any (start <= hour_value < end for start, end in conveyance_times):
            noise_level = round(np.random.uniform(60.0, 80.0), 2)
        else: 
            noise_level = round(np.random.uniform(20.0, 50.0), 2)

        if use_caps and noise_level > noise_cap:
            noise_level = noise_cap

        noise_level = round(noise_level, 2)

        simulated_data.append({
            "timestamp": timestamp,
            "temperature": round(temperature, 2),
            "noise_level": noise_level
        })

simulated_df = pd.DataFrame(simulated_data)

print(simulated_df.head())

            timestamp  temperature  noise_level
0 2024-08-01 08:00:00        60.98        42.31
1 2024-08-01 09:00:00        66.56        59.00
2 2024-08-01 10:00:00        71.55        42.84
3 2024-08-01 11:00:00        75.44        48.31
4 2024-08-01 12:00:00        78.97        59.00


#### Save to CSV

In [12]:
simulated_df.to_csv('shipdock_environmental_data_0809.csv', index=False)